# Modelo de PLI para resolução do problema de corte

## Instalação do OR-Tools

In [31]:
%pip install ortools

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Importação da biblioteca

In [32]:
from ortools.linear_solver import pywraplp

## Funções auxiliares

### Função auxiliar: encontro das possíveis combinações

In [33]:
def encontrar_combinacoes(comprimentos, limite):
    """
    Gera todas as combinações possíveis de cortes respeitando o limite.
    
    Args:
        comprimentos (list[int]): Tamanhos disponíveis para corte.
        limite (int): Comprimento máximo da barra.

    Returns:
        set: Um conjunto de tuplas representando combinações válidas.
              Cada tupla contém:
              - Quantidade de cada comprimento
              - Total utilizado
              - Desperdício
    """
    solucoes = set()

    def backtrack(combinacao, total):
        if total > limite:
            return

        pode_adicionar = any(total + comp <= limite for comp in comprimentos)

        if not pode_adicionar and total > 0:
            desperdicio = limite - total
            solucoes.add(tuple(combinacao + [total, desperdicio]))
            return

        for i, comp in enumerate(comprimentos):
            nova_combinacao = combinacao[:]
            nova_combinacao[i] += 1
            backtrack(nova_combinacao, total + comp)

    combinacao_inicial = [0] * len(comprimentos)
    backtrack(combinacao_inicial, 0)

    return solucoes


### Função auxilir: print das soluções

In [34]:
def printar_solucoes(solucoes):
    """
    Exibe todas as soluções possíveis de corte e seus desperdícios.

    Args:
        solucoes (set): Conjunto de soluções gerado por `encontrar_combinacoes`.
    """
    for i, s in enumerate(solucoes): 
        total = s[-2]
        desperdicio = s[-1]
        partes = []
        for qtd, tamanho in zip(s[:-2], comprimentos):
            if qtd > 0:
                partes.append(f"p{i+1}: {qtd} barras de {tamanho} metros")
        frase = " + ".join(partes)
        print(f"\n{frase} | total = {total} metros | desperdicio = {desperdicio} metros")


### Função auxiliar: Montando os desperdícios

In [35]:
def montar_desperdicio(solucoes):
    """
    Extrai apenas os desperdícios das soluções.

    Args:
        solucoes (set): Conjunto de soluções.

    Returns:
        list[int]: Lista com os desperdícios correspondentes a cada solução.
    """
    return [s[-1] for s in solucoes]


### Função auxiliar: transpor a matriz

In [36]:
def transpor_matriz(matriz):
    """
    Transpõe uma matriz (linha vira coluna).

    Args:
        matriz (list[list[int]]): Matriz original.

    Returns:
        list[list[int]]: Matriz transposta.
    """
    linhas = len(matriz)
    colunas = len(matriz[0])
    transposta = [[0 for _ in range(linhas)] for _ in range(colunas)]
    for i in range(linhas):
        for j in range(colunas):
            transposta[j][i] = matriz[i][j]
    return transposta


### Função auxiliar: Montagem da matriz de solução

In [37]:
def montar_matriz_respostas(solucoes):
    """
    Monta uma matriz onde cada linha representa um tipo de corte e cada coluna uma solução.

    Args:
        solucoes (set): Conjunto de soluções.

    Returns:
        list[list[int]]: Matriz transposta com os coeficientes para o PLI.
    """
    return transpor_matriz([list(s[:-2]) for s in solucoes])


## Leitura e entrada do arquivo de input

In [38]:
arquivo = 'entrada.txt'

comprimentos = []
limitantes = []

with open(arquivo, 'r') as f:
    primeira_linha = f.readline().strip()
    limite, n = map(int, primeira_linha.split())

    for _ in range(n):
        linha = f.readline().strip()
        comp, lim = map(int, linha.split())
        comprimentos.append(comp)
        limitantes.append(lim)

# Exibe os dados lidos
print("Limite:", limite)
print("Comprimentos:", comprimentos)
print("Limitantes:", limitantes)

Limite: 150
Comprimentos: [50, 60, 80]
Limitantes: [120, 100, 70]


## Geração das combinações possíveis de corte

In [39]:
solucoes = encontrar_combinacoes(comprimentos, limite)

desperdicio = montar_desperdicio(solucoes)

n = len(desperdicio)

coeficientes = montar_matriz_respostas(solucoes)

## Montagem do Modelo de Programação Linear Inteira(PLI)

In [40]:
solver = pywraplp.Solver.CreateSolver('SCIP') #declara o solver: SCIP para PLI e GLOP para PL
infinity = solver.infinity()

###  Variáveis de decisão: número de vezes que cada solução será usada

In [41]:
x = []
for i in range(len(coeficientes[0])):
    x.append(solver.IntVar(0.0, infinity, 'p' + str(i+1)))

### Função objetivo: minimizar desperdício total


In [42]:
objetivo = solver.Objective()
for i in range(n):
  objetivo.SetCoefficient(x[i], desperdicio[i])
objetivo.SetMinimization()

###  Restrições: atender a demanda mínima de cada tipo de peça

In [43]:
for i in range(len(coeficientes)):
  restricao = solver.Constraint(limitantes[i],infinity)
  for j in range(len(coeficientes[i])):
    restricao.SetCoefficient(x[j], coeficientes[i][j])

## Solução do problema

In [44]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
  print('Solução:')
  print('Valor da função objetivo =', solver.Objective().Value())
  for i in range(n):
    print(x[i].name(), '=', x[i].solution_value())
printar_solucoes(solucoes)

Solução:
Valor da função objetivo = 1000.0
p1 = 0.0
p2 = 0.0
p3 = 0.0
p4 = 100.0
p5 = 40.0

p1: 2 barras de 60 metros | total = 120 metros | desperdicio = 30 metros

p2: 1 barras de 50 metros + p2: 1 barras de 60 metros | total = 110 metros | desperdicio = 40 metros

p3: 1 barras de 50 metros + p3: 1 barras de 80 metros | total = 130 metros | desperdicio = 20 metros

p4: 1 barras de 60 metros + p4: 1 barras de 80 metros | total = 140 metros | desperdicio = 10 metros

p5: 3 barras de 50 metros | total = 150 metros | desperdicio = 0 metros


## Exportação do modelo

In [45]:
print(solver.ExportModelAsLpFormat(False))

\ Generated by MPModelProtoExporter
\   Name             : 
\   Format           : Free
\   Constraints      : 3
\   Variables        : 5
\     Binary         : 0
\     Integer        : 5
\     Continuous     : 0
Minimize
 Obj: +30 p1 +40 p2 +20 p3 +10 p4 
Subject to
 auto_c_000000000: +1 p2 +1 p3 +3 p5  >= 120
 auto_c_000000001: +2 p1 +1 p2 +1 p4  >= 100
 auto_c_000000002: +1 p3 +1 p4  >= 70
Bounds
 0 <= p1 <= inf
 0 <= p2 <= inf
 0 <= p3 <= inf
 0 <= p4 <= inf
 0 <= p5 <= inf
Generals
 p1
 p2
 p3
 p4
 p5
End

